<a href="https://colab.research.google.com/github/davidpedem/davidpedem/blob/main/Copia_de_Copia_de_explicacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ejercicios Adicionales
#David Peña de Miguel

# Creando un modelo de traducción

Objetivos de la práctica:
- Conocer cómo crear un modelo de traducción de texto usando la librería transformers.

Para este notebook es conveniente que compruebes que la opción de GPU está activada (Runtime -> Change Runtime Type).

Para esta práctica tendrás que recuperar tu cuenta de HuggingFace y tu token de escritura.

## Ejercicios adicionales

1. Utiliza un dataset de traducción distinto al de neutralización (2 puntos).
- He utilizado el dataset hackathon-somos-nlp-2023/informes_discriminacion_gitana
2. Utiliza un modelo distinto al de mbart (puedes usar modelos como t5 o pegasus) (2 puntos).
- He utilizado modelo google/flan-t5-large
3. Usando lo que veremos en la última sesión del curso, construye un dataset para traducción y construye un modelo que lo utilice (1 punto).

Si haces estas tareas debes crear un nuevo notebook de Colaboratory (una para cada una de ellas) y guardarlo en tu repositorio de GitHub.

## Instalando librerías

Por defecto, el entorno de Google Colab no tiene instaladas las librerías de [HuggingFace](https://huggingface.co/), por lo que vamos a hacer en primer lugar es instalar las librerías: [Transformers](https://huggingface.co/docs/transformers/index), [Datasets](https://huggingface.co/docs/datasets/index), y [Evaluate](https://huggingface.co/docs/evaluate/index).
- Librería Transformers: para entrenar modelos de procesado de lenguaje natural
- Libreía Datasets: permite acceder de manera sencilla a distintos datasets de audio, visión y  procesado de lenguaje natural disponibles en Hugging Face.
- Librería Evaluate: sirve para evaluar modelos de aprendizaje automático

In [ ]:
!pip install datasets evaluate transformers[sentencepiece] sacrebleu accelerate -Uq

A continuación nos conectamos al hub de huggingface, lo que nos permitirá subir nuestros modelos a este entorno. Al ejecutar la siguiente celda aparecerá un widget en el cual tendremos que copiar el token generado en el primer paso y pulsar en el botón login.

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

## Dataset

En el ejemplo anterior visto en clase utilizamos el [Spanish Gender Neutralization
dataset](https://huggingface.co/datasets/hackathon-pln-es/neutral-es) que contiene un dataset para traducir de español "normal" a español "inclusivo".

En este caso he decidido utilizar el [Informe de discriminación gitana](https://huggingface.co/datasets/hackathon-somos-nlp-2023/informes_discriminacion_gitana). Un dataset en el que se presentan distintas situaciones discriminatorias acontecidas por el pueblo gitano.
Comenzamos descargando el dataset. Se modifican dichas oraciones a un castellano no discriminatorio.

In [ ]:
from datasets import load_dataset

In [ ]:
dataset = load_dataset("hackathon-somos-nlp-2023/informes_discriminacion_gitana")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Veamos el contenido de este dataset.

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['sintetico', 'text', 'intervencion', 'tipo_discriminacion', 'resultado'],
        num_rows: 1791
    })
    test: Dataset({
        features: ['sintetico', 'text', 'intervencion', 'tipo_discriminacion', 'resultado'],
        num_rows: 100
    })
    valid: Dataset({
        features: ['sintetico', 'text', 'intervencion', 'tipo_discriminacion', 'resultado'],
        num_rows: 99
    })
})

Podemos ver que tenemos un objeto DatasetDict que puede verse como un diccionario. Dicho diccionario contiene un atributo `train` y un atributo `test`, por lo que en este caso ya disponemos de la división para entrenar nuestro modelo. Vamos a ver alguna de las frases del dataset, para lo que tenemos que acceder al atributo `train`.

In [ ]:
dataset['train']

Dataset({
    features: ['sintetico', 'text', 'intervencion', 'tipo_discriminacion', 'resultado'],
    num_rows: 1791
})

Con el anterior comando vemos que tenemos un Dataset con tres columnas: `train`, `test` y `valid`. Si queremos ver el contenido del dataset, lo podemos transformar a formato pandas y verlo como una tabla. Vemos a continuación que cada fila de nuestro dataset contiene una frase y una frase alineada que está neutralizada.

In [ ]:
dataset['train'].to_pandas()

,sintetico,text,intervencion,tipo_discriminacion,resultado
0,1,Una joven contactó con una empresa que había p...,Se le informó a la joven de sus derechos como ...,Discriminación directa,Negativo.
1,0,"Un hombre gitano fue a una empresa, a propuest...",Se informó al joven de sus derechos a la igual...,Discriminación directa,Neutro.
2,0,Tras ver una oferta de trabajo en una frutería...,Contactamos por teléfono con la dueña de la fr...,Discriminación directa,Negativo.
3,0,Una mujer acudió a su oficina bancaria para so...,La pareja presentó una hoja de reclamaciones a...,Discriminación directa,Negativo.
4,0,Una joven gitana fue a hacer la compra acompañ...,"La participante, que había recibido una formac...",Discriminación directa,Neutro.
...,...,...,...,...,...
1786,0,El periódico Norte Expres publicó una noticia ...,Se envió una carta de queja al periódico y se ...,Discriminación directa,Neutro.
1787,0,Dos mujeres gitanas fueron a un supermercado a...,Desde la FSG informamos a las mujeres de sus d...,Discriminación interseccional,Neutro.
1788,0,La tía de un niño menor de edad nos envió un i...,Desde la FSG enviamos una carta a la orientado...,Discriminación directa,Neutro.
1789,0,"Un hombre gitano se encontraba en su coche, ap...",La víctima nos envía por medios electrónicos e...,Discriminación directa,Neutro.


Para poder entrenar un modelo con este dataset es necesario tokenizarlo. Cada modelo tokeniza de una manera distinta, por lo que es necesario indicar el modelo para tokenizar el texto. En nuestro caso vamos a utilizar un modelo llamado [t5](https://huggingface.co/google/flan-t5-large).

In [ ]:
from transformers import AutoTokenizer #Adaptamos el modelo a una tarea concreta

model_checkpoint = "google/flan-t5-large" #Construimos el tokenizador a partir del modelo t5 (en el ejercicio anterior utilizamos mbart)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Definimos una función para tokenizar el texto. Notar que para otros datasets
será necesario cambiar el valor de "gendered" y "neutral" por las columnas que queramos
tokenizar, el resto del código no hará falta tocarlo.

In [ ]:
max_length = 128


def preprocess_function(examples):
    model_inputs = tokenizer(
        examples["train"],
        max_length=max_length,
        truncation=True,
    )
    labels = tokenizer(
        examples["test"], max_length=max_length, truncation=True
    )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

Tokenizamos el dataset y lo mostramos.

In [ ]:
tokenized_datasets = dataset.map(preprocess_function, batched=True)
tokenized_datasets

Map:   0%|          | 0/1791 [00:00<?, ? examples/s]

KeyError: 'train'

In [ ]:
tokenized_datasets["train"].to_pandas() #41 es el token de comienzo de frase y [2] es el componente final
#Attention_mask dice en qué palabras debo fijarme: las que tienen información y cuáles no.

,source,target,input_ids,attention_mask,labels
0,give me the cities in virginia .,"(city(A),loc(A,B),const(B,stateid(virginia)))","[428, 140, 8, 3119, 16, 24556, 23, 9, 3, 5, 1]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[41, 6726, 599, 188, 201, 5133, 599, 188, 6, 2..."
1,what are the high points of states surrounding...,"(high_point(B,A),loc(A,B),state(B),next_to(B,C...","[125, 33, 8, 306, 979, 13, 2315, 3825, 3041, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[41, 6739, 834, 2700, 599, 279, 6, 188, 201, 5..."
2,name the rivers in arkansas .,"(river(A),loc(A,B),const(B,stateid(arkansas)))","[564, 8, 16912, 16, 3, 6604, 3247, 9, 7, 3, 5, 1]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[41, 5927, 49, 599, 188, 201, 5133, 599, 188, ..."
3,name all the rivers in colorado .,"(river(A),loc(A,B),const(B,stateid(colorado)))","[564, 66, 8, 16912, 16, 945, 9, 26, 32, 3, 5, 1]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[41, 5927, 49, 599, 188, 201, 5133, 599, 188, ..."
4,can you tell me the capital of texas ?,"(capital(A),loc(A,B),const(B,stateid(texas)))","[54, 25, 817, 140, 8, 1784, 13, 3, 10354, 9, 7...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[41, 4010, 9538, 599, 188, 201, 5133, 599, 188..."
...,...,...,...,...,...
875,which states have cities named austin ?,"(state(A),loc(B,A),city(B),const(B,cityid(aust...","[84, 2315, 43, 3119, 2650, 403, 17, 77, 3, 58, 1]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[41, 5540, 599, 188, 201, 5133, 599, 279, 6, 1..."
876,which states have points higher than the highe...,"(state(A),loc(B,A),place(B),higher(B,C),highes...","[84, 2315, 43, 979, 1146, 145, 8, 2030, 500, 1...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[41, 5540, 599, 188, 201, 5133, 599, 279, 6, 1..."
877,which states have points that are higher than ...,"(state(A),loc(B,A),higher(B,C),highest(C,(plac...","[84, 2315, 43, 979, 24, 33, 1146, 145, 8, 2030...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...","[41, 5540, 599, 188, 201, 5133, 599, 279, 6, 1..."
878,which states lie on the largest river in the u...,"(state(A),traverse(B,A),largest(B,(river(B),lo...","[84, 2315, 11761, 30, 8, 2015, 4033, 16, 8, 18...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[41, 5540, 599, 188, 201, 1313, 7583, 599, 279..."


In [ ]:
len(tokenized_datasets["train"]["attention_mask"][23]) #Nos tenemos que fijar en los primeros 13 elementos, el resto son 0s

13

Al igual que ocurría con los modelos de clasificación de texto, podemos ver que han aparecido tres nuevas columnas ('input_ids', 'labels' y 'attention_mask') que serán utilizadas para entrenar el modelo. Notar que ya disponemos de una columna labels que será la que tendrá que predecir el modelo.  

Por último, antes de definir nuestro modelo tenemos que definir una función que se va a encargar de preparar los datos para que sean procesados de manera eficiente por el modelo.

In [ ]:
from transformers import DataCollatorForSeq2Seq #Prepara los datos para pasarlos al modelo

data_collator = DataCollatorForSeq2Seq(tokenizer)

## Modelo

Pasamos ahora a definir el modelo, lo primero que vamos a definir son los argumentos con los que vamos a entrenar nuestro modelo.

In [ ]:
from transformers import Seq2SeqTrainingArguments


args = Seq2SeqTrainingArguments(
    # Carpeta donde se guardará el modelo
    output_dir="dvitel/geo",
    # Evaluar el modelo cada época
    evaluation_strategy="epoch",
    # Valores para modificar los pesos del modelo
    learning_rate=5.6e-5,
    weight_decay=0.01,
    # Número de ejemplos que se muestran cada vez al modelo
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    # Número de veces que que se muestran todos los datos al modelo
    num_train_epochs=2,
    # Realizar predicciones al validar
    predict_with_generate=True,
    push_to_hub=True,
)

A continuación definimos nuestro modelo, para ello usamos la clase `AutoModelForSeq2SeqLM` y vamos a utilizar un modelo pre-entrenado (recordar lo que era el transfer learning). Para ello solo tenemos que indicar el nombre de nuestro modelo (definido previamente en la variable `model_checkpoint`.  

In [ ]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Ahora definimos la función que usaremos para calcular la precisión de nuestro modelo. En este caso usaremos la métrica [sacrebleu](https://huggingface.co/spaces/evaluate-metric/sacrebleu) que nos proporciona la métrica bleu utilizada en el Workshop on Machine Translation (WMT).

In [ ]:
import numpy as np #Definimos una métrica de evaluación: calcula cómod de buena es una traducción  con respecto a una traducción de referencia

import evaluate

metric = evaluate.load("sacrebleu")

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result


Ya podemos definir nuestro objeto `trainer` que usaremos para entrenar nuestro modelo (en este caso será un modelo secuencia a secuencia). La estructura de este objeto será siempre la misma. Le tenemos que proporcionar:
1. El modelo.
2. La configuración del entrenamiento.
3. El conjunto de entrenamiento.
4. El conjunto de test.
5. El objeto que prepara los datos.
6. El tokenizador.
7. La métrica.

In [ ]:
from transformers import Seq2SeqTrainer

trainer = Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

KeyError: 'test'

Y ahora entrenamos el modelo mediante el método `train`. Este proceso es algo más lento que el entrenamiento de los modelos de clasificación y puede llevar unos 10 minutos.

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,No log,0.048993,96.265900,19.010400
2,0.246200,0.022013,98.213200,18.541700


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200, 'early_stopping': True, 'num_beams': 5, 'forced_eos_token_id': 2}


TrainOutput(global_step=880, training_loss=0.15103775696320967, metrics={'train_runtime': 661.6365, 'train_samples_per_second': 10.619, 'train_steps_per_second': 1.33, 'total_flos': 519539733135360.0, 'train_loss': 0.15103775696320967, 'epoch': 2.0})

## Compartiendo el modelo

Una vez que tenemos entrenado nuestro modelo, nos interesa compartirlo con el resto del mundo para que puedan usarlo y también compararlo con otros modelos.

Es por ello que vamos a subir nuestro modelo al hub de huggingface. Para ello tenemos que ejecutar el siguiente comando.

In [ ]:
%cd mbart-neutralization
trainer.push_to_hub(commit_message="Training complete", tags="simplification")

Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'max_length': 200, 'early_stopping': True, 'num_beams': 5, 'forced_eos_token_id': 2}


/content/mbart-neutralization


model.safetensors:   0%|          | 0.00/2.44G [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/davidpedem/mbart-neutralization/commit/c5322a9e3080df6b137a6bd65f846f8d799d82f2', commit_message='Training complete', commit_description='', oid='c5322a9e3080df6b137a6bd65f846f8d799d82f2', pr_url=None, pr_revision=None, pr_num=None)

Al terminar de ejecutarse el comando anterior tendremos nuestro modelo disponible en https://huggingface.co/davidpedem/mbart-neutralization (en la URL anterior deberás reemplazar joheras por tu nombre de usuario).

Si accedes al enlace anterior, verás que tienes tu modelo disponible y una [tarjeta de modelo (o *model card*)](https://huggingface.co/docs/hub/model-cards) con una breve descripción del mismo. Es conveniente que proporciones información adicional a la *model card* ya que la que se genera de forma automática es demasiado básica.

Además verás que en el enlace anterior tienes un pequeño widget que te permite hacer predicciones con tu modelo.

Finalmente, vamos a ver cómo usar nuestro modelo para hacer predicciones desde código (esto puede ser útil sí por ejemplo nos interesa procesar múltiples textos de manera secuencial).


## Usando el modelo

En este caso al ser un modelo que hemos entrenado nosotros mismos podríamos usar los ficheros locales, pero vamos a ver cómo usar el modelo que acabamos de subir al hub de HuggingFace.

Para ello usamos un `pipeline` al que le debemos indicar el nombre del modelo que queremos descargar.

In [ ]:
from transformers import pipeline
neutralizer = pipeline('text2text-generation', model='davidpedem/mbart-neutralization')

Ahora podemos hacer traducciones con nuestro modelo.

In [ ]:
neutralizer('El profesor de la asignatura decidirá cómo hacer el examen') #No se puede meter cualquier frase

[{'generated_text': 'El profesor de la asignatura decidirá cómo hacer el examen'}]

In [ ]:
neutralizer('Realizamos entrevistas con los encargados de la clase') #Solo se pueden introducir aquellas frases que estén en el modelo

[{'generated_text': 'Realizamos entrevistas con el personal responsable de la clase'}]